# [NTDS'19] assignment 1: network science
[ntds'19]: https://github.com/mdeff/ntds_2019

[Eda Bayram](https://lts4.epfl.ch/bayram), [EPFL LTS4](https://lts4.epfl.ch) and
[Nikolaos Karalias](https://people.epfl.ch/nikolaos.karalias), [EPFL LTS2](https://lts2.epfl.ch).

## Students

* Team: 8
* Students: Adrian Villarroel (for the indivudual submission) or Ariel Alba, Andres Montero, Elias Poroma, Adrian Villarroel (for the team submission)

## Rules

Grading:
* The first deadline is for individual submissions. The second deadline is for the team submission.
* All team members will receive the same grade based on the team solution submitted on the second deadline.
* As a fallback, a team can ask for individual grading. In that case, solutions submitted on the first deadline are graded.
* Collaboration between team members is encouraged. No collaboration between teams is allowed.

Submission:
* Textual answers shall be short. Typically one to two sentences.
* Code has to be clean.
* You cannot import any other library than we imported.
  Note that Networkx is imported in the second section and cannot be used in the first.
* When submitting, the notebook is executed and the results are stored. I.e., if you open the notebook again it should show numerical results and plots. We won't be able to execute your notebooks.
* The notebook is re-executed from a blank state before submission. That is to be sure it is reproducible. You can click "Kernel" then "Restart Kernel and Run All Cells" in Jupyter.

## Objective

The purpose of this milestone is to explore a given dataset, represent it by network by constructing different graphs. In the first section, you will analyze the network properties. In the second section, you will explore various network models and find out the network model fitting the ones you construct from the dataset.

## Cora Dataset

The [Cora dataset](https://linqs.soe.ucsc.edu/node/236) consists of scientific publications classified into one of seven research fields. 

* **Citation graph:** the citation network can be constructed from the connections given in the `cora.cites` file.
* **Feature graph:** each publication in the dataset is described by a 0/1-valued word vector indicating the absence/presence of the corresponding word from the dictionary and its research field, given in the `cora.content` file. The dictionary consists of 1433 unique words. A feature graph can be constructed using the Euclidean distance between the feature vector of the publications.

The [`README`](data/cora/README) provides details about the content of [`cora.cites`](data/cora/cora.cites) and [`cora.content`](data/cora/cora.content).

## Section 1: Network Properties

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

%matplotlib inline

### Question 1: Construct a Citation Graph and a Feature Graph

Read the `cora.content` file into a Pandas DataFrame by setting a header for the column names. Check the `README` file.

In [ ]:
column_list = ["paper_id"]+["w"+str(x) for x in range(1,1434)]+["class_label"] # Your code here
pd_content = pd.read_csv('data/cora/cora.content', delimiter='\t', names=column_list) 
pd_content.head()

Print out the number of papers contained in each of the reasearch fields.

**Hint:** You can use the `value_counts()` function.

In [ ]:
pd.value_counts(pd_content["class_label"]) # Your code here.

Select all papers from a field of your choice and store their feature vectors into a NumPy array.
Check its shape.

In [ ]:
my_field = "Neural_Networks" # Your code here.
features = pd_content[pd_content["class_label"] == my_field].iloc[:,1:pd_content.shape[1]-1].values # Your code here.
features.shape

Let $D$ be the Euclidean distance matrix whose $(i,j)$ entry corresponds to the Euclidean distance between feature vectors $i$ and $j$.
Using the feature vectors of the papers from the field which you have selected, construct $D$ as a Numpy array. 

In [ ]:
# (Added by Adrian) Test numpy broadcast for Euclidean distance
v1 = np.asarray([2,1,3])
v2 = np.asarray([6,5,7])
v3 = np.asarray([2,3,4])
mat = np.asarray([v1,v2,v3])
print(mat)
d_mat = np.linalg.norm(mat-mat[:,np.newaxis,:],axis=2)
print(d_mat)
print(np.linalg.norm(v1-v2))
print(np.linalg.norm(v1-v3))
print(np.linalg.norm(v2-v3))

In [ ]:
distance = np.linalg.norm(features-features[:,np.newaxis,:],axis=2) # Your code here.
distance.shape

Check the mean pairwise distance $\mathbb{E}[D]$.

In [ ]:
mean_distance = distance.mean()
mean_distance

Plot an histogram of the euclidean distances.

In [ ]:
plt.figure(1, figsize=(8, 4))
plt.title("Histogram of Euclidean distances between papers")
plt.hist(distance.flatten());

Now create an adjacency matrix for the papers by thresholding the Euclidean distance matrix.
The resulting (unweighted) adjacency matrix should have entries
$$ A_{ij} = \begin{cases} 1, \; \text{if} \; d(i,j)< \mathbb{E}[D], \; i \neq j, \\ 0, \; \text{otherwise.} \end{cases} $$

First, let us choose the mean distance as the threshold.

In [ ]:
threshold = mean_distance
A_feature = np.where(distance < threshold, 1, 0) - np.eye(distance.shape[0]) # Your code here.

Now read the `cora.cites` file and construct the citation graph by converting the given citation connections into an adjacency matrix.

In [ ]:
cora_cites = np.genfromtxt('data/cora/cora.cites', delimiter='\t')

index = np.vectorize(dict(zip(pd_content["paper_id"],pd_content["paper_id"].index)).get)(cora_cites) # Your code here
A_citation = np.zeros((pd_content.shape[0],pd_content.shape[0])) # Your code here
A_citation[index[:,0],index[:,1]] = 1 # Your code here
A_citation.shape

Get the adjacency matrix of the citation graph for the field that you chose.
You have to appropriately reduce the adjacency matrix of the citation graph.

In [ ]:
# (Added by Adrian) Test np.ix_()
a = np.asarray([[1,2,3,4],[5,6,7,8],[9,10,11,12],[13,14,15,16]])
b = a[np.ix_(np.asarray([1,3]),np.asarray([0,3]))]
print(b)

In [ ]:
my_f_index = pd_content[pd_content["class_label"] == my_field].index.values.astype(np.intp) # Your code here
A_citation= A_citation[np.ix_(my_f_index,my_f_index)] # Your code here.

Check if your adjacency matrix is symmetric. Symmetrize your final adjacency matrix if it's not already symmetric.

In [ ]:
A_citation = .5 * (A_citation+A_citation.T) # Your code here. Question: Is this symmetrization correct?
# A_citation = A_citation.T@A_citation
np.count_nonzero(A_citation - A_citation.transpose())

In [ ]:
# (Added by Adrian) Check values from citation matrix
np.unique(A_citation)

Check the shape of your adjacency matrix again.

In [ ]:
A_citation.shape

### Question 2: Degree Distribution and Moments

What is the total number of edges in each graph?

In [ ]:
num_edges_feature = np.sum(A_feature) / 2 # Your code here. It is undirected
num_edges_citation = np.sum(A_citation) # Your code here. It is directed
print(f"Number of edges in the feature graph: {num_edges_feature}")
print(f"Number of edges in the citation graph: {num_edges_citation}")

Plot the degree distribution histogram for each of the graphs.

In [ ]:
# (Added by Adrian) Show degree calculation simple example
test_a_citation = np.asarray([[0,1,1],[0,0,1],[1,0,0]])
print(test_a_citation)
print("K_out = " + str(np.sum(test_a_citation,axis = 1)))
print("K_in = " + str(np.sum(test_a_citation,axis = 0)))
print("K = " + str(np.sum(test_a_citation,axis = 0)+np.sum(test_a_citation,axis = 1)))

In [ ]:
degrees_citation = np.sum(A_citation,axis = 0) + np.sum(A_citation,axis = 1) # Your code here. It is directed
degrees_feature = np.sum(A_feature,axis = 0) # Your code here. It is undirected

deg_hist_normalization = np.ones(degrees_citation.shape[0]) / degrees_citation.shape[0]

fig, axes = plt.subplots(1, 2, figsize=(16, 4))
axes[0].set_title('Citation graph degree distribution')
axes[0].hist(degrees_citation, weights=deg_hist_normalization);
axes[1].set_title('Feature graph degree distribution')
axes[1].hist(degrees_feature, weights=deg_hist_normalization);

Calculate the first and second moments of the degree distribution of each graph.

In [ ]:
cit_moment_1 = np.mean(degrees_citation) # Your code here.
cit_moment_2 = np.var(degrees_citation) # Your code here.

feat_moment_1 = np.mean(degrees_feature) # Your code here.
feat_moment_2 = np.var(degrees_feature) # Your code here.

print(f"1st moment of citation graph: {cit_moment_1}") 
print(f"2nd moment of citation graph: {cit_moment_2}") 
print(f"1st moment of feature graph: {feat_moment_1}") 
print(f"2nd moment of feature graph: {feat_moment_2}") 

What information do the moments provide you about the graphs?
Explain the differences in moments between graphs by comparing their degree distributions.

**Your answer here:** The first moment is the average number of nodes connected to a node and the second moment is the variance. The citation degree distribution has all of its points concentrated in the range 0-6.5, that is why the average and the variance are low, whereas points of the feature distribution are distributed in the range 100-800 and highly concentrated in the range 100-300 resulting in a higher average and variance.

Select the 20 largest hubs for each of the graphs and remove them. Observe the sparsity pattern of the adjacency matrices of the citation and feature graphs before and after such a reduction.

In [ ]:
n = 20 # Your code here
index_hubs_A_feature = (-degrees_feature).argsort()[:n] # Your code here
index_hubs_A_citation = (-degrees_citation).argsort()[:n] # Your code here
reduced_A_feature = np.delete(np.delete(A_feature,index_hubs_A_feature,axis = 0),index_hubs_A_feature, axis = 1) # Your code here
reduced_A_citation = np.delete(np.delete(A_citation,index_hubs_A_citation,axis = 0),index_hubs_A_citation, axis = 1) # Your code here

fig, axes = plt.subplots(2, 2, figsize=(16, 16))
axes[0, 0].set_title('Feature graph: adjacency matrix sparsity pattern')
axes[0, 0].spy(A_feature);
axes[0, 1].set_title('Feature graph without top 20 hubs: adjacency matrix sparsity pattern')
axes[0, 1].spy(reduced_A_feature);
axes[1, 0].set_title('Citation graph: adjacency matrix sparsity pattern')
axes[1, 0].spy(A_citation);
axes[1, 1].set_title('Citation graph without top 20 hubs: adjacency matrix sparsity pattern')
axes[1, 1].spy(reduced_A_citation);

Plot the new degree distribution histograms.

In [ ]:
reduced_degrees_feat = np.sum(reduced_A_feature,axis = 0) # Your code here.
reduced_degrees_cit = np.sum(reduced_A_citation,axis = 0) + np.sum(reduced_A_citation,axis = 1) # Your code here.

deg_hist_normalization = np.ones(reduced_degrees_feat.shape[0])/reduced_degrees_feat.shape[0]

fig, axes = plt.subplots(1, 2, figsize=(16, 4))
axes[0].set_title('Citation graph degree distribution')
axes[0].hist(reduced_degrees_cit, weights=deg_hist_normalization);
axes[1].set_title('Feature graph degree distribution')
axes[1].hist(reduced_degrees_feat, weights=deg_hist_normalization);

Compute the first and second moments for the new graphs.

In [ ]:
reduced_cit_moment_1 = np.mean(reduced_degrees_cit) # Your code here.
reduced_cit_moment_2 = np.var(reduced_degrees_cit) # Your code here.

reduced_feat_moment_1 = np.mean(reduced_degrees_feat) # Your code here.
reduced_feat_moment_2 = np.var(reduced_degrees_feat) # Your code here.

print("Citation graph first moment:", reduced_cit_moment_1)
print("Citation graph second moment:", reduced_cit_moment_2)
print("Feature graph first moment: ", reduced_feat_moment_1)
print("Feature graph second moment: ", reduced_feat_moment_2)

Print the number of edges in the reduced graphs.

In [ ]:
reduced_num_edges_feature = np.sum(reduced_A_feature) / 2 # Your code here. It is undirected
reduced_num_edges_citation = np.sum(reduced_A_citation) # Your code here. It is directed
print(f"Number of edges in the reduced feature graph: {reduced_num_edges_feature}") # Your code here.
print(f"Number of edges in the reduced citation graph: {reduced_num_edges_citation}") # Your code here.

Is the effect of removing the hubs the same for both networks? Look at the percentage changes for each moment. Which of the moments is affected the most and in which graph? Explain why.  

**Hint:** Examine the degree distributions.

**Your answer here:** The effect is larger for the citation network because it has few nodes that have a degree significantly larger from the average (hubs). The second moment in the citation graph is affected the most because, according to the degree distribution, there was a small portion of nodes having degrees higher than 8 which accounted for most of the variance.

In [ ]:
delta_cit_moment_1 = (cit_moment_1 - reduced_cit_moment_1) *100 / cit_moment_1 # Your code here
delta_cit_moment_2 = (cit_moment_2 - reduced_cit_moment_2) *100 / cit_moment_2 # Your code here

delta_feat_moment_1 = (feat_moment_1 - reduced_feat_moment_1) *100 / feat_moment_1 # Your code here
delta_feat_moment_2 = (feat_moment_2 - reduced_feat_moment_2) *100 / feat_moment_2 # Your code here

print("Citation moment 1 percentage change: " + str(delta_cit_moment_1) +" %") # Your code here
print("Feature moment 1 percentage change: " + str(delta_feat_moment_1) +" %") # Your code here
print("Citation moment 2 percentage change: " + str(delta_cit_moment_2 ) +" %") # Your code here
print("Feature moment 2 percentage change: " + str(delta_feat_moment_2 ) +" %") # Your code here

### Question 3: Pruning, sparsity, paths

By adjusting the threshold of the euclidean distance matrix, prune the feature graph so that its number of edges is roughly close (within a hundred edges) to the number of edges in the citation graph.

In [ ]:
mean_distance = distance.mean()

# We take the new treshold as mean_distance*0.5
threshold = mean_distance*0.5# Your code here.

# We create the A feature pruned with the new treshhold
A_feature_pruned = np.where(distance < threshold, 1, 0) - np.eye(distance.shape[0])  # Your code here

#The number of edges is equal to the total values of the matrix /2 , is an undirected graph
num_edges_feature_pruned = 0.5*np.sum(A_feature_pruned)# Your code here.

print(f"Number of edges in the feature graph: {num_edges_feature}")
print(f"Number of edges in the feature graph after pruning: {num_edges_feature_pruned}")
print(f"Number of edges in the citation graph: {num_edges_citation}")

Check your results by comparing the sparsity patterns and total number of edges between the graphs.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 6))
axes[0].set_title('Citation graph sparsity')
axes[0].spy(A_citation);
axes[1].set_title('Feature graph sparsity')
axes[1].spy(A_feature_pruned);

Let $C_{k}(i,j)$ denote the number of paths of length $k$ from node $i$ to node $j$. 

We define the path matrix $P$, with entries
$ P_{ij} = \displaystyle\sum_{k=0}^{N}C_{k}(i,j). $

Calculate the path matrices for both the citation and the unpruned feature graphs for $N =10$.  

**Hint:** Use [powers of the adjacency matrix](https://en.wikipedia.org/wiki/Adjacency_matrix#Matrix_powers).

In [ ]:
A_citation.shape
# A_feature.shape
path_matrix_citation = np.zeros((818,818))

In [ ]:
# Your code here.
path_matrix_citation = np.zeros((818,818))
path_matrix_feature = np.zeros((818,818))

for i in range(10+1):
    path_matrix_citation += np.linalg.matrix_power(A_citation,i)
    path_matrix_feature += np.linalg.matrix_power(A_feature,i)
    

Check the sparsity pattern for both of path matrices.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 9))
axes[0].set_title('Citation Path matrix sparsity')
axes[0].spy(path_matrix_citation);
axes[1].set_title('Feature Path matrix sparsity')
axes[1].spy(path_matrix_feature);

Now calculate the path matrix of the pruned feature graph for $N=10$. Plot the corresponding sparsity pattern. Is there any difference?

In [ ]:
path_matrix_pruned = np.linalg.matrix_power(A_feature_pruned,10)# Your code here.
for i in range(10+1):
    path_matrix_pruned += np.linalg.matrix_power(A_feature_pruned,i)
plt.figure(figsize=(12, 6))
plt.title('Feature Path matrix sparsity')
plt.spy(path_matrix_pruned);

**Your answer here:**

Describe how you can use the above process of counting paths to determine whether a graph is connected or not. Is the original (unpruned) feature graph connected?

For a graph to be connected, all the pairs of vertices need to be connected at least for a given row, by taking the Adjancecy matrix to the power, we will get a non-zero value only if both entries are non_zero, meaning the i,j value of the matrix are connected. 
With this explanation we can conclude that feature graph unpruned is connected 

**Your answer here:** 

If the graph is connected, how can you guess its diameter using the path matrix?

We can find the diameter of the graph when A^k has non-zero values, meaning that all the paths are connected up until k (I+ A^1,....+ A^k, therefore the diameter is k

**Your answer here:**

If any of your graphs is connected, calculate the diameter using that process.

In [ ]:
def find_diameter(matrix):
    """Function to calculate the diameter of a graph,
    Diameter = I + A^1 + .... + A^k up until there are no 0, meaning every 
    vertex connects with every vertex for distance K"""
    degree = 0
    path_matrix = np.zeros(matrix.shape)
    i=0
    path_matrix += np.linalg.matrix_power(matrix,i)
    while 0 in path_matrix:
        i += 1
        path_matrix += np.linalg.matrix_power(matrix,i)
        
        if i > 20:
            print("infinite graph!")
            return "infinite"
    return i


In [ ]:
diameter = find_diameter(A_feature) # Your code here.
print(f"The diameter is: {diameter}")

Check if your guess was correct using [NetworkX](https://networkx.github.io/documentation/stable/reference/algorithms/generated/networkx.algorithms.distance_measures.diameter.html).
Note: usage of NetworkX is only allowed in this part of Section 1.

In [ ]:
import networkx as nx
feature_graph = nx.from_numpy_matrix(A_feature)
print(f"Diameter according to networkx: {nx.diameter(feature_graph)}")

## Section 2: Network Models

In this section, you will analyze the feature and citation graphs you constructed in the previous section in terms of the network model types.
For this purpose, you can use the NetworkX libary imported below.

In [ ]:
import networkx as nx

Let us create NetworkX graph objects from the adjacency matrices computed in the previous section.

In [ ]:
G_citation = nx.from_numpy_matrix(A_citation)
print('Number of nodes: {}, Number of edges: {}'. format(G_citation.number_of_nodes(), G_citation.number_of_edges()))
print('Number of self-loops: {}, Number of connected components: {}'. format(G_citation.number_of_selfloops(), nx.number_connected_components(G_citation)))

In the rest of this assignment, we will consider the pruned feature graph as the feature network.

In [ ]:
G_feature = nx.from_numpy_matrix(A_feature_pruned)
print('Number of nodes: {}, Number of edges: {}'. format(G_feature.number_of_nodes(), G_feature.number_of_edges()))
print('Number of self-loops: {}, Number of connected components: {}'. format(G_feature.number_of_selfloops(), nx.number_connected_components(G_feature)))

### Question 4: Simulation with Erdős–Rényi and Barabási–Albert models

Create an Erdős–Rényi and a Barabási–Albert graph using NetworkX to simulate the citation graph and the feature graph you have. When choosing parameters for the networks, take into account the number of vertices and edges of the original networks.

The number of nodes should exactly match the number of nodes in the original citation and feature graphs.

In [ ]:
assert len(G_citation.nodes()) == len(G_feature.nodes())
n = len(G_citation.nodes())
n

The number of match shall fit the average of the number of edges in the citation and the feature graph.

In [ ]:
m = np.round((G_citation.size() + G_feature.size()) / 2)
m

How do you determine the probability parameter for the Erdős–Rényi graph?

**Your answer here:**

In [ ]:
p = # Your code here.
G_er = nx.erdos_renyi_graph(n, p)

Check the number of edges in the Erdős–Rényi graph.

In [ ]:
print('My Erdos-Rényi network that simulates the citation graph has {} edges.'.format(G_er.size()))

How do you determine the preferential attachment parameter for Barabási–Albert graphs?

**Your answer here:**

In [ ]:
q = # Your code here.
G_ba = nx.barabasi_albert_graph(n, q)

Check the number of edges in the Barabási–Albert graph.

In [ ]:
print('My Barabási-Albert network that simulates the citation graph has {} edges.'.format(G_ba.size()))

### Question 5: Giant Component

Check the size of the largest connected component in the citation and feature graphs.

In [ ]:
giant_citation = # Your code here.
print('The giant component of the citation graph has {} nodes and {} edges.'.format(giant_citation.number_of_nodes(), giant_citation.size()))

In [ ]:
giant_feature = # Your code here.
print('The giant component of the feature graph has {} nodes and {} edges.'.format(giant_feature.number_of_nodes(), giant_feature.size()))

Check the size of the giant components in the generated Erdős–Rényi graph.

In [ ]:
giant_er = # Your code here.
print('The giant component of the Erdos-Rényi network has {} nodes and {} edges.'.format(giant_er.number_of_nodes(), giant_er.size()))

Let us match the number of nodes in the giant component of the feature graph by simulating a new Erdős–Rényi network.
How do you choose the probability parameter this time? 

**Hint:** Recall the expected giant component size from the lectures.

**Your answer here:**

In [ ]:
p_new = # Your code here.
G_er_new = nx.erdos_renyi_graph(n, p_new)

Check the size of the new Erdős–Rényi network and its giant component.

In [ ]:
print('My new Erdos Renyi network that simulates the citation graph has {} edges.'.format(G_er_new.size()))
giant_er_new = # Your code here.
print('The giant component of the new Erdos-Rényi network has {} nodes and {} edges.'.format(giant_er_new.number_of_nodes(), giant_er_new.size()))

### Question 6: Degree Distributions

Recall the degree distribution of the citation and the feature graph.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 6))
axes[0].set_title('Citation graph')
citation_degrees = # Your code here.
axes[0].hist(citation_degrees);
axes[1].set_title('Feature graph')
feature_degrees = # Your code here.
axes[1].hist(feature_degrees);

What does the degree distribution tell us about a network? Can you make a prediction on the network model type of the citation and the feature graph by looking at their degree distributions?

**Your answer here:**

Now, plot the degree distribution historgrams for the simulated networks.

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(20, 6))
axes[0].set_title('Erdos-Rényi network')
er_degrees = # Your code here.
axes[0].hist(er_degrees);
axes[1].set_title('Barabási-Albert network')
ba_degrees = # Your code here.
axes[1].hist(ba_degrees);
axes[2].set_title('new Erdos-Rényi network')
er_new_degrees = # Your code here.
axes[2].hist(er_new_degrees);

In terms of the degree distribution, is there a good match between the citation and feature graphs and the simulated networks?
For the citation graph, choose one of the simulated networks above that match its degree distribution best. Indicate your preference below.

**Your answer here:** 

You can also simulate a network using the configuration model to match its degree disctribution exactly. Refer to [Configuration model](https://networkx.github.io/documentation/stable/reference/generated/networkx.generators.degree_seq.configuration_model.html#networkx.generators.degree_seq.configuration_model).

Let us create another network to match the degree distribution of the feature graph. 

In [ ]:
G_config = nx.configuration_model(feature_degrees) 
print('Configuration model has {} nodes and {} edges.'.format(G_config.number_of_nodes(), G_config.size()))

Does it mean that we create the same graph with the feature graph by the configuration model? If not, how do you understand that they are not the same?

**Your answer here:** 

### Question 7: Clustering Coefficient

Let us check the average clustering coefficient of the original citation and feature graphs. 

In [ ]:
nx.average_clustering(G_citation)

In [ ]:
nx.average_clustering(G_feature)

What does the clustering coefficient tell us about a network? Comment on the values you obtain for the citation and feature graph.

**Your answer here:**

Now, let us check the average clustering coefficient for the simulated networks.

In [ ]:
nx.average_clustering(G_er)

In [ ]:
nx.average_clustering(G_ba)

In [ ]:
nx.average_clustering(nx.Graph(G_config))

Comment on the values you obtain for the simulated networks. Is there any good match to the citation or feature graph in terms of clustering coefficient?

**Your answer here:**

Check the other [network model generators](https://networkx.github.io/documentation/networkx-1.10/reference/generators.html) provided by NetworkX. Which one do you predict to have a better match to the citation graph or the feature graph in terms of degree distribution and clustering coefficient at the same time? Justify your answer.

**Your answer here:**

If you find a better fit, create a graph object below for that network model. Print the number of edges and the average clustering coefficient. Plot the histogram of the degree distribution.

In [ ]:
# Your code here.

Comment on the similarities of your match.

**Your answer here:**